In [803]:
import math
import numpy as np
import csv 
import operator
from random import sample
import matplotlib.pyplot
from matplotlib import pyplot as plt

In [804]:
TrainingPercent = 80 #Percentage of data considered for training the model
ValidationPercent = 10 #Percentage of data considered for validation
TestPercent = 10 #Percentage of data considered for testing
C_Lambda = 0.03 #term used for regularization in order to prevent overfitting

def generateRawData(filename) : 
    raw_data = open(filename, 'r')
    reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
    next(reader)
    x = list(reader)
    data = np.array(x)
    return data

In [805]:
# Reading the matrix and random shuffling of the datasets
human_observed_dataset = generateRawData('HumanObserved-Features-Data.csv')
print(type(human_observed_dataset))
np.array(np.random.shuffle(human_observed_dataset))
print(type(human_observed_dataset))
diffn_pairs_data = generateRawData('diffn_pairs.csv')
np.array(np.random.shuffle(diffn_pairs_data))
gsc_diffn_pair = generateRawData('GSC-Features-Data/diffn_pairs.csv')
np.array(np.random.shuffle(gsc_diffn_pair))
same_pairs_data = generateRawData('same_pairs.csv')
np.array(np.random.shuffle(same_pairs_data))
gsc_same_pair = generateRawData('GSC-Features-Data/same_pairs.csv')
np.array(np.random.shuffle(gsc_same_pair))
gsc_dataset = generateRawData('GSC-Features-Data/GSC-Features.csv')
np.array(np.random.shuffle(gsc_dataset))

print(human_observed_dataset.shape)
print(diffn_pairs_data.shape)
print(same_pairs_data.shape)
print(gsc_dataset.shape)
print(gsc_diffn_pair.shape)
print(gsc_same_pair.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1026, 11)
(293032, 3)
(791, 3)
(14072, 513)
(762557, 3)
(71531, 3)


In [806]:
img_id = []
gsc_img_id = []
feature_vector = []
same_pairs_target = []
diff_pairs_target = []
total_target = []
gsc_feature_vector = []
########### Human Observed #################
feature_vector = human_observed_dataset[:,2: ]
feature_vector = [[int(y) for y in x] for x in feature_vector]
############## GSC ########################
gsc_feature_vector = gsc_dataset[:,1: ]
gsc_feature_vector = [[int(y) for y in x] for x in gsc_feature_vector]
print('GSC ' + str(len(gsc_feature_vector)))
print('GSC length ' + str(len(gsc_feature_vector[0])))

GSC 14072
GSC length 512


In [807]:
########### Human Observed #################
same_pairs_target = same_pairs_data[:,-1]
same_pairs_target = same_pairs_target.astype(np.int_)
print('Same pair target shape is : '+ str(len(same_pairs_target)))

diff_pairs_target = diffn_pairs_data[0:791,-1]
diff_pairs_target = diff_pairs_target.astype(np.int_)
print('Diff pair target shape is : '+ str(len(diff_pairs_target)))

total_target = np.concatenate((same_pairs_target, diff_pairs_target))
print('Total target shape is : '+ str(len(total_target)))
print('Total target type is : '+ str(type(total_target)))
print(total_target)
############## GSC ########################
gsc_same_pairs_target = gsc_same_pair[0:2000,-1]
gsc_same_pairs_target = gsc_same_pairs_target.astype(np.int_)
print('GSC Same pair target shape is : '+ str(len(gsc_same_pairs_target)))

gsc_diff_pairs_target = gsc_diffn_pair[0:2000,-1]
gsc_diff_pairs_target = gsc_diff_pairs_target.astype(np.int_)
print('GSC Diff pair target shape is : '+ str(len(gsc_diff_pairs_target)))

gsc_total_target = np.concatenate((gsc_same_pairs_target, gsc_diff_pairs_target))
print('GSC Total target shape is : '+ str(len(gsc_total_target)))
print('GSC Total target type is : '+ str(type(gsc_total_target)))
print(gsc_total_target)

Same pair target shape is : 791
Diff pair target shape is : 791
Total target shape is : 1582
Total target type is : <class 'numpy.ndarray'>
[1 1 1 ... 0 0 0]
GSC Same pair target shape is : 2000
GSC Diff pair target shape is : 2000
GSC Total target shape is : 4000
GSC Total target type is : <class 'numpy.ndarray'>
[1 1 1 ... 0 0 0]


In [808]:
img_id = []
gsc_img_id = []
########### Human Observed #################
with open('HumanObserved-Features-Data.csv', 'r') as rf:
    read = csv.reader(rf)
    next(read)
    for i in read:
        img_id.extend([i[1]])

############## GSC ########################
with open('GSC-Features-Data/GSC-Features.csv', 'r') as grf:
    gread = csv.reader(grf)
    next(gread)
    for j in gread:
        gsc_img_id.extend([j[0]])
    print(str(len(gsc_img_id)))

14072


In [809]:
########### Human Observed #################    
a=[]
b=[]
same_pair_concatenation=[]
same_pair_subtraction = []
diff_pair_concateantion=[]
diff_pair_subtraction = []
total_human_concat_data =[]
total_human_subtract_data=[]
with open('same_pairs.csv', 'r') as fi:
    same_read = csv.reader(fi)
    next(same_read)
    for i in same_read:
        a = feature_vector[img_id.index(i[0])]
        b = feature_vector[img_id.index(i[1])]
        same_pair_subtraction.append(list(map(operator.sub, a, b)))
        same_pair_concatenation.append(a+b)   
print(type(same_pair_concatenation))
print('Same pair concatenation shape is : '+str(len(same_pair_concatenation)))
print('Same pair Subtraction shape is : '+ str(len(same_pair_subtraction)))

############## GSC ########################
count = 0
ga=[]
gb=[]
gsc_same_pair_concatenation=[]
gsc_same_pair_subtraction = []
gsc_total_human_concat_data =[]
gsc_total_human_subtract_data=[]
with open('GSC-Features-Data/same_pairs.csv', 'r') as gfi:
    gsame_read = csv.reader(gfi)
    next(gsame_read)
    for i in gsame_read:
        count+=1
        ga = gsc_feature_vector[gsc_img_id.index(i[0])]
        gb = gsc_feature_vector[gsc_img_id.index(i[1])]
        gsc_same_pair_subtraction.append(list(map(operator.sub, ga, gb)))
        gsc_same_pair_concatenation.append(ga+gb)
        if count == 2000 :
            break
print(type(same_pair_concatenation))
print('GSC Same pair concatenation shape is : '+str(len(gsc_same_pair_concatenation)))
print('GSC Same pair concatenation shape is : '+str(len(gsc_same_pair_concatenation[0])))
print('GSC Same pair Subtraction shape is : '+ str(len(gsc_same_pair_subtraction)))


<class 'list'>
Same pair concatenation shape is : 791
Same pair Subtraction shape is : 791
<class 'list'>
GSC Same pair concatenation shape is : 2000
GSC Same pair concatenation shape is : 1024
GSC Same pair Subtraction shape is : 2000


In [810]:
gsc_diff_pair_concateantion=[]
gsc_diff_pair_subtraction = []
########### Human Observed #################  
with open('diffn_pairs.csv', 'r') as fi:
    diff_read = csv.reader(fi)
    next(diff_read)
    for j in diff_read:
        a = feature_vector[img_id.index(j[0])]
        b = feature_vector[img_id.index(j[1])]
        diff_pair_concateantion.append(a+b)
        diff_pair_subtraction.append(list(map(operator.sub, a, b)))
diff_pair_concateantion = diff_pair_concateantion[:791]
diff_pair_subtraction = diff_pair_subtraction[:791]
print(type(diff_pair_concateantion))
print('Diff pair Subtraction shape is : '+str(len(diff_pair_subtraction)))
print('Diff pair concatenation shape is : '+str(len(diff_pair_concateantion)))

############## GSC ########################
count = 0
with open('GSC-Features-Data/diffn_pairs.csv', 'r') as gfi:
    gdiff_read = csv.reader(gfi)
    next(gdiff_read)
    for j in gdiff_read:
        count+=1
        ga = gsc_feature_vector[gsc_img_id.index(j[0])]
        gb = gsc_feature_vector[gsc_img_id.index(j[1])]
        gsc_diff_pair_concateantion.append(ga+gb)
        gsc_diff_pair_subtraction.append(list(map(operator.sub, ga, gb)))
        if count == 2000 :
            break
print(type(diff_pair_concateantion))
print('GSC Diff pair Subtraction shape is : '+str(len(gsc_diff_pair_concateantion)))
print('GSC Diff pair concatenation shape is : '+str(len(gsc_diff_pair_subtraction)))

<class 'list'>
Diff pair Subtraction shape is : 791
Diff pair concatenation shape is : 791
<class 'list'>
GSC Diff pair Subtraction shape is : 2000
GSC Diff pair concatenation shape is : 2000


In [811]:
########### Human Observed #################  
total_human_concat_data = same_pair_concatenation + diff_pair_concateantion
total_human_concat_data = np.transpose(total_human_concat_data) 
print('NO. of Rows in Total Human Concatenated Data : ' + str(len(total_human_concat_data)))
print('NO. of Columns in Total Human Concatenated Data : ' + str(len(total_human_concat_data[0])))
total_human_subtract_data = same_pair_subtraction + diff_pair_subtraction
total_human_subtract_data = np.transpose(total_human_subtract_data)
print('NO. of Rows in Total Human Subtracted Data : ' + str(len(total_human_subtract_data)))
print('NO. of Columns in Total Human Subtracted Data : ' + str(len(total_human_subtract_data[0])))

############## GSC ########################
gsc_total_human_concat_data =[]
gsc_total_human_subtract_data=[]

gsc_total_human_concat_data = gsc_same_pair_concatenation + gsc_diff_pair_concateantion
gsc_total_human_concat_data = np.transpose(gsc_total_human_concat_data) 
print('NO. of Rows in Total GSC Concatenated Data : ' + str(len(gsc_total_human_concat_data)))
print('NO. of Columns in Total GSC Concatenated Data : ' + str(len(gsc_total_human_concat_data[0])))

gsc_total_human_subtract_data = gsc_same_pair_subtraction + gsc_diff_pair_subtraction
gsc_total_human_subtract_data = np.transpose(gsc_total_human_subtract_data)
print('NO. of Rows in Total GSC Subtracted Data : ' + str(len(gsc_total_human_subtract_data)))
print('NO. of Columns in Total GSC Subtracted Data : ' + str(len(gsc_total_human_subtract_data[0])))



NO. of Rows in Total Human Concatenated Data : 18
NO. of Columns in Total Human Concatenated Data : 1582
NO. of Rows in Total Human Subtracted Data : 9
NO. of Columns in Total Human Subtracted Data : 1582
NO. of Rows in Total GSC Concatenated Data : 1024
NO. of Columns in Total GSC Concatenated Data : 4000
NO. of Rows in Total GSC Subtracted Data : 512
NO. of Columns in Total GSC Subtracted Data : 4000


In [813]:
#Prepare trainig data
# This is taking 80% of the target values from the entire target 
def GenerateTrainingTarget (rawTraining,TrainingPercent):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t  
# This method is used to specify and seperate 80% of the data is for training data
def GenerateTrainingDataMatrix(rawData, TrainingPercent):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

In [814]:
TrainingTarget = np.array(GenerateTrainingTarget(total_target,TrainingPercent))
GSCTrainingTarget = np.array(GenerateTrainingTarget(gsc_total_target,TrainingPercent))
TrainingDataForConcatenationHuman   = GenerateTrainingDataMatrix(total_human_concat_data,TrainingPercent)
TrainingDataForConcatenationGSC   = GenerateTrainingDataMatrix(gsc_total_human_concat_data,TrainingPercent)
TrainingDataForSubtractionHuman   = GenerateTrainingDataMatrix(total_human_subtract_data,TrainingPercent)
TrainingDataForSubtractionGSC   = GenerateTrainingDataMatrix(gsc_total_human_subtract_data,TrainingPercent)
print(TrainingTarget.shape)
print(GSCTrainingTarget.shape)
print(TrainingDataForConcatenationHuman.shape)
print(TrainingDataForSubtractionHuman.shape)
print(TrainingDataForConcatenationGSC.shape)
print(TrainingDataForSubtractionGSC.shape)

(1266,)
(3200,)
(18, 1266)
(9, 1266)
(1024, 3200)
(512, 3200)


In [815]:
# This method is used to specify and seperate 10% of the data is for validation data.
# The same method is called for testing data as well because it is the same 10% again.
def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix
# This method is for the file QueryLevelNormt.csv
# prepares the target values for validation dataset
def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

In [816]:
# genertae validation data and its target values
# genertae validation data and its target values
ValDataAct = np.array(GenerateValTargetVector(total_target,ValidationPercent, (len(TrainingTarget))))
GSCVAlDataAct = np.array(GenerateValTargetVector(gsc_total_target,ValidationPercent, (len(GSCTrainingTarget))))
ValDataForHumanConcatenation    = GenerateValData(total_human_concat_data,ValidationPercent, (len(TrainingTarget)))
ValDataForGSCConcatenation    = GenerateValData(gsc_total_human_concat_data,ValidationPercent, (len(GSCTrainingTarget)))
ValDataForSubtractionHuman   = GenerateValData(total_human_subtract_data,ValidationPercent, (len(TrainingTarget)))
ValDataForSubtractionGSC   = GenerateValData(gsc_total_human_subtract_data,ValidationPercent, (len(GSCTrainingTarget)))
print(ValDataAct.shape)
print(GSCVAlDataAct.shape)
print(ValDataForHumanConcatenation.shape)
print(ValDataForSubtractionHuman.shape)
print(ValDataForGSCConcatenation.shape)
print(ValDataForSubtractionGSC.shape)

(158,)
(399,)
(18, 158)
(9, 158)
(1024, 399)
(512, 399)


In [817]:
# genertae testing data and its target values
TestDataAct = np.array(GenerateValTargetVector(total_target,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
GSCTestDataAct = np.array(GenerateValTargetVector(gsc_total_target,TestPercent, (len(GSCTrainingTarget)+len(GSCVAlDataAct))))
TestDataForConcatenationHuman = GenerateValData(total_human_concat_data,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
TestDataForGSCConcatenation    = GenerateValData(gsc_total_human_concat_data,TestPercent, (len(GSCTrainingTarget)+len(GSCVAlDataAct)))
TestDataForSubtractionHuman   = GenerateValData(total_human_subtract_data,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
TestDataForSubtractionGSC   = GenerateValData(gsc_total_human_subtract_data,TestPercent, (len(GSCTrainingTarget)+len(GSCVAlDataAct)))
print(TestDataAct.shape)
print(GSCTestDataAct.shape)
print(TestDataForConcatenationHuman.shape)
print(TestDataForSubtractionHuman.shape)
print(TestDataForGSCConcatenation.shape)
print(TestDataForSubtractionGSC.shape)

(157,)
(399,)
(18, 157)
(9, 157)
(1024, 399)
(512, 399)


In [818]:
# this function is used to generate the bigsigma matrix which is used in the normal distribution formula.
# Big sigma is the variance matrix. 
# We assume all varainces are equal for a particular feature.
# Hence the variance matrix will be of the shape 41 * 41.
# it is not 46 * 46 because 5 features are being deleted.
# we ll have elements only in the diagnol of the matrix.
def GenerateBigSigma(Data, MuMatrix,TrainingPercent):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))
    varVect     = []
    for i in range(0,(DataT.shape[1])):
        vct = []
        for j in range(0,int(TrainingLen) ):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]+0.2
    ##print ("BigSigma Generated..")
    return BigSigma

# The gaussian distribution function is 0.5(X-Mu)* BigSigInv* (x-Mu)transpose.
def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow) # This subtracts X- Mu and stores in R
    T = np.dot(BigSigInv,np.transpose(R))  # Multiplies R transpose and BigSigInv and stores in T
    L = np.dot(R,T) # Multiplies T and R and returns L
    return L
# This functions is the gaussian distribution formula.
# This is used to return each PHI and not the PHI matrix.
def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x
# This method is used to get the Phi matrix which is the design matrix.
# Phi matrix is the result of the radial basis function.
# We are using 10 radial basis function and hence dimension of phi matrix will be 10 * number of data
def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01 ))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma) # We calculated BigSigma only. But we needed Inverse of Big Sigma acc. to the formula
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)#Every PHI calculated is stored in a Matrix called PHI
    #print ("PHI Generated..")
    return PHI

# Lamba is a regularization term used to prevent overfitting
# This method is used to get the closed form W* 
def GetWeightsClosedForm(PHI, T, Lambda):
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    print (INTER.shape)
    print (T.shape)
    W           = np.dot(INTER, T)
    ##print ("Training Weights Generated..")
    return W

# This is for the regression formula y = phi *w transpose
# returns our predicted value
def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y
# This is used to find our accuracy and Erms based on target and the predicted value.
def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

In [819]:
from sklearn.cluster import KMeans
def skeletonForLinear(TrainingDataForConcatenationHuman, total_human_concat_data, TrainingPercent,TrainingTarget,C_Lambda,TestDataForConcatenationHuman,ValDataForHumanConcatenation):
    kmeans = KMeans(n_clusters=20, random_state=0).fit(np.transpose(TrainingDataForConcatenationHuman))
    # We have to select one representative from each cluster.
    # we are choosing the center of each cluster to be the representative.
    # This is also known as the feature points.
    Mu = kmeans.cluster_centers_
    print(Mu.shape)
    print(total_human_concat_data.shape)
    BigSigma     = GenerateBigSigma(total_human_concat_data, Mu, TrainingPercent)
    TRAINING_PHI = GetPhiMatrix(total_human_concat_data, Mu, BigSigma, TrainingPercent)
    W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget, (C_Lambda)) 
    #Before Calculating Validation/Testing error, we need to create validation/Testing Design matrix.
    # We cannot use the same Training Data design matrix.
    TEST_PHI     = GetPhiMatrix(TestDataForConcatenationHuman, Mu, BigSigma, 100) 
    VAL_PHI      = GetPhiMatrix(ValDataForHumanConcatenation, Mu, BigSigma, 100)
    print(BigSigma.shape)
    print(TRAINING_PHI.shape)
    print(W.shape)
    print(VAL_PHI.shape)
    print(TEST_PHI.shape)
    return TRAINING_PHI,W,TEST_PHI,VAL_PHI

In [820]:
TRAINING_PHI,W,TEST_PHI,VAL_PHI = skeletonForLinear(TrainingDataForConcatenationHuman, total_human_concat_data, TrainingPercent,TrainingTarget,C_Lambda,TestDataForConcatenationHuman,ValDataForHumanConcatenation)
SUB_TRAINING_PHI,SUB_W,SUB_TEST_PHI,SUB_VAL_PHI = skeletonForLinear(TrainingDataForSubtractionHuman, total_human_subtract_data, TrainingPercent,TrainingTarget,C_Lambda,TestDataForSubtractionHuman,ValDataForSubtractionHuman)

(20, 18)
(18, 1582)
(20, 1266)
(1266,)
(18, 18)
(1266, 20)
(20,)
(158, 20)
(157, 20)
(20, 9)
(9, 1582)
(20, 1266)
(1266,)
(9, 9)
(1266, 20)
(20,)
(158, 20)
(157, 20)


In [821]:
GSC_TRAINING_PHI,GSC_W,GSC_TEST_PHI,GSC_VAL_PHI = skeletonForLinear(TrainingDataForConcatenationGSC, gsc_total_human_concat_data, TrainingPercent,GSCTrainingTarget,C_Lambda,TestDataForGSCConcatenation,ValDataForGSCConcatenation)
GSC_SUB_TRAINING_PHI,GSC_SUB_W,GSC_SUB_TEST_PHI,GSC_SUB_VAL_PHI = skeletonForLinear(TrainingDataForSubtractionGSC, gsc_total_human_subtract_data, TrainingPercent,GSCTrainingTarget,C_Lambda,TestDataForSubtractionGSC,ValDataForSubtractionGSC)

(20, 1024)
(1024, 4000)
(20, 3200)
(3200,)
(1024, 1024)
(3200, 20)
(20,)
(399, 20)
(399, 20)
(20, 512)
(512, 4000)
(20, 3200)
(3200,)
(512, 512)
(3200, 20)
(20,)
(399, 20)
(399, 20)


In [822]:
def skeletonForSgd(TRAINING_PHI,W,TEST_PHI,VAL_PHI,ValDataAct,TestDataAct,TrainingTarget):
    # The w* obtained from previous method is multiplied with 220 a scalar value
    # This is because SGD  first takes a random initial value w(0)
    W_Now        = np.dot(3, W)
    # La is the lambda which as stated earlier is used as a regularizer in order to prevent overfitting
    La           = 0.2 
    learningRate = 0.01
    # Learning rate in SGD must not be too high as it might never converge or if it is too low it will take lot of time to converge
    L_Erms_Val   = []
    L_Erms_TR    = []
    L_Erms_Test  = []

    for i in range(0,200):
        # Number of data considered in the entire dataset
        # The next 4 steps are based on the formula where we calculate the weights based on previous iteration.
        # They are used to adjust the weights based on previous value.
        #print ('---------Iteration: ' + str(i) + '--------------')
        Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
        La_Delta_E_W  = np.dot(La,W_Now)
        Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
        Delta_W       = -np.dot(learningRate,Delta_E)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next # W_Now will be the updated weight now

        #-----------------TrainingData Accuracy---------------------#
        TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
        Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
        L_Erms_TR.append(float(Erms_TR.split(',')[1]))

        #-----------------ValidationData Accuracy---------------------#
        VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
        Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
        L_Erms_Val.append(float(Erms_Val.split(',')[1]))

        #-----------------TestingData Accuracy---------------------#
        TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
        Erms_Test = GetErms(TEST_OUT,TestDataAct)
        L_Erms_Test.append(float(Erms_Test.split(',')[1]))
        
        

   
    
    return min(L_Erms_TR),min(L_Erms_Val),min(L_Erms_Test)

In [823]:
L_Erm,L_Erms_Val,Erms = skeletonForSgd(TRAINING_PHI,W,TEST_PHI,VAL_PHI,ValDataAct,TestDataAct,TrainingTarget)
print ('----------Gradient Descent Solution For Human Observed Data Concatenated--------------------')
print ("Accuracy Training   = " + str(np.around((L_Erm),5)))
print ("Accuracy Validation = " + str(np.around((L_Erms_Val),5)))
print ("Accuracy Testing    = " + str(np.around((Erms),5)))
L_ERM,Erms_Test,L_Erms_Test = skeletonForSgd(SUB_TRAINING_PHI,SUB_W,SUB_TEST_PHI,SUB_VAL_PHI,ValDataAct,TestDataAct,TrainingTarget)
print ('----------Gradient Descent Solution For Human Observed Data Subtracted--------------------')
print ("Accuracy Training   = " + str(np.around((L_ERM),5)))
print ("Accuracy Validation = " + str(np.around((Erms_Test),5)))
print ("Accuracy Testing    = " + str(np.around((L_Erms_Test),5)))


----------Gradient Descent Solution For Human Observed Data Concatenated--------------------
Accuracy Training   = 0.7307
Accuracy Validation = 0.76307
Accuracy Testing    = 0.59972
----------Gradient Descent Solution For Human Observed Data Subtracted--------------------
Accuracy Training   = 0.77775
Accuracy Validation = 0.8452
Accuracy Testing    = 0.88312


In [824]:
L_Erms_TR,L_Erms_Val,L_Erms_Test = skeletonForSgd(GSC_TRAINING_PHI,GSC_W,GSC_TEST_PHI,GSC_VAL_PHI,GSCVAlDataAct,GSCTestDataAct,GSCTrainingTarget)
print ('----------Gradient Descent Solution For GSC Data Concatenated--------------------')
print ("Accuracy Training   = " + str(np.around((L_Erms_TR),5)))
print ("Accuracy Validation = " + str(np.around((L_Erm),5)))
print ("Accuracy Testing    = " + str(np.around((L_ERM),5)))
L_Erms_TR,L_Erms_Val,L_Erms_Test = skeletonForSgd(GSC_SUB_TRAINING_PHI,GSC_SUB_W,GSC_SUB_TEST_PHI,GSC_SUB_VAL_PHI,GSCVAlDataAct,GSCTestDataAct,GSCTrainingTarget)
print ('----------Gradient Descent Solution For GSC Data Subtracted--------------------')
print ("Accuracy Training   = " + str(np.around((L_Erms_TR),5)))
print ("Accuracy Validation = " + str(np.around((Erms),5)))
print ("Accuracy Testing    = " + str(np.around((Erms_Test),5)))

----------Gradient Descent Solution For GSC Data Concatenated--------------------
Accuracy Training   = 0.79057
Accuracy Validation = 0.7307
Accuracy Testing    = 0.77775
----------Gradient Descent Solution For GSC Data Subtracted--------------------
Accuracy Training   = 0.79057
Accuracy Validation = 0.59972
Accuracy Testing    = 0.8452


In [825]:
def LogisticRegression(TrainingDataForConcatenationHuman,total_human_concat_data,TrainingTarget):
    Wlogistic=[]
    learningRateLogistic = 0.03
    Wlogistic = np.random.random((len(TrainingDataForConcatenationHuman),len(TrainingDataForConcatenationHuman)))
    Wlogistic = Wlogistic[1,:]

    for i in range(300) :
        ysiginput = np.dot(Wlogistic,TrainingDataForConcatenationHuman)
        sigoutputa =  1 / (1 + np.exp(-ysiginput))
        Gradient =  np.dot(TrainingDataForConcatenationHuman,np.transpose((sigoutputa-ysiginput)))/len(TrainingTarget)
        Wlogistic = Wlogistic + learningRateLogistic * Gradient
        predicted = np.dot(Wlogistic,TrainingDataForConcatenationHuman)
        Accuracy =  1 / (1 + np.exp(-predicted)).mean()*100
    return np.around((Accuracy),2)

In [826]:
TrainingAccuracy = LogisticRegression(TrainingDataForConcatenationHuman,total_human_concat_data,TrainingTarget)
print(TrainingAccuracy)
ValidationAccuracy = LogisticRegression((ValDataForHumanConcatenation),total_human_concat_data,(ValDataAct))
print(ValidationAccuracy)
TestingAccuracy =LogisticRegression((TestDataForConcatenationHuman),total_human_concat_data,(TestDataAct))
print(TestingAccuracy)

TrainingAccuracy = LogisticRegression(TrainingDataForSubtractionHuman,total_human_subtract_data,TrainingTarget)
print(TrainingAccuracy)
ValidationAccuracy = LogisticRegression((ValDataForSubtractionHuman),total_human_subtract_data,(ValDataAct))
print(ValidationAccuracy)
TestingAccuracy =LogisticRegression((TestDataForSubtractionHuman),total_human_subtract_data,(TestDataAct))
print(TestingAccuracy)

65.53
65.84
65.84
53.28
61.59
63.08


In [794]:
TrainingAccuracy = LogisticRegression(TrainingDataForConcatenationGSC,gsc_total_human_concat_data,GSCTrainingTarget)
ValidationAccuracy = LogisticRegression((ValDataForGSCConcatenation),gsc_total_human_concat_data,(GSCVAlDataAct))
TestingAccuracy =LogisticRegression((TestDataForGSCConcatenation),gsc_total_human_concat_data,(GSCTestDataAct))

TrainingAccuracy = LogisticRegression(TrainingDataForSubtractionGSC,gsc_total_human_subtract_data,GSCTrainingTarget)
print(TrainingAccuracy)
ValidationAccuracy = LogisticRegression((ValDataForSubtractionGSC),gsc_total_human_subtract_data,(GSCVAlDataAct))
print(ValidationAccuracy)
TestingAccuracy =LogisticRegression((TestDataForSubtractionGSC),gsc_total_human_subtract_data,(GSCTestDataAct))
print(TestingAccuracy)

54.59
60.17
60.31


In [792]:
################Neural Networks ################
#######half completed- need to find the best suited hyperparameter######
def NN(total_human_concat_data,total_target) :
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Dense, Activation, Dropout
    from keras.callbacks import EarlyStopping, TensorBoard
    import numpy
    # fix random seed for reproducibility
    seed = 7
    validation_data_split = 0.2
    tb_batch_size = 10
    early_patience = 100
    numpy.random.seed(seed)

    # split into input (X) and output (Y) variables
    X = np.transpose(total_human_concat_data)
    print(len(X))
    print(len(X[0]))
    Y = total_target

    tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
    earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')

    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=len(X[0]), init='uniform', activation='relu'))
    model.add(Dense(200, init='uniform', activation='relu'))
    model.add(Dense(300, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    history=model.fit(X, Y, nb_epoch=10000, batch_size=5,  verbose=2, validation_split=validation_data_split, callbacks = [tensorboard_cb,earlystopping_cb])
    # calculate predictions
    predictions = model.predict(X)


In [793]:
NN(total_human_concat_data,total_target)
NN(total_human_subtract_data,total_target)

1582
18


C:\Users\tceve\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, input_dim=18, activation="relu", kernel_initializer="uniform")`
C:\Users\tceve\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_initializer="uniform")`
C:\Users\tceve\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(300, activation="relu", kernel_initializer="uniform")`
C:\Users\tceve\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
C:\Users\tceve\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1265 samples, validate on 317 samples
Epoch 1/10000
 - 1s - loss: 0.3228 - acc: 0.8585 - val_loss: 10.1007 - val_acc: 0.0820
Epoch 2/10000
 - 0s - loss: 0.1496 - acc: 0.9486 - val_loss: 14.2870 - val_acc: 0.0820
Epoch 3/10000
 - 0s - loss: 0.1044 - acc: 0.9684 - val_loss: 14.6381 - val_acc: 0.0820
Epoch 4/10000
 - 0s - loss: 0.0711 - acc: 0.9779 - val_loss: 14.6389 - val_acc: 0.0820
Epoch 5/10000
 - 0s - loss: 0.0505 - acc: 0.9850 - val_loss: 14.6382 - val_acc: 0.0820
Epoch 6/10000
 - 0s - loss: 0.0352 - acc: 0.9913 - val_loss: 14.6396 - val_acc: 0.0820
Epoch 7/10000
 - 0s - loss: 0.0452 - acc: 0.9866 - val_loss: 14.6350 - val_acc: 0.0820
Epoch 8/10000
 - 0s - loss: 0.0358 - acc: 0.9905 - val_loss: 14.5783 - val_acc: 0.0820
Epoch 9/10000
 - 0s - loss: 0.0331 - acc: 0.9897 - val_loss: 14.6370 - val_acc: 0.0820
Epoch 10/10000
 - 0s - loss: 0.0185 - acc: 0.9960 - val_loss: 14.6350 - val_acc: 0.0820
Epoch 11/10000
 - 0s - loss: 0.0072 - acc: 0.9992 - val_loss: 14.6352 - val_acc: 0

Epoch 94/10000
 - 0s - loss: 0.0017 - acc: 0.9992 - val_loss: 14.6348 - val_acc: 0.0820
Epoch 95/10000
 - 0s - loss: 8.2900e-04 - acc: 1.0000 - val_loss: 14.6348 - val_acc: 0.0820
Epoch 96/10000
 - 0s - loss: 0.0024 - acc: 0.9984 - val_loss: 14.6348 - val_acc: 0.0820
Epoch 97/10000
 - 0s - loss: 0.0017 - acc: 0.9992 - val_loss: 14.6348 - val_acc: 0.0820
Epoch 98/10000
 - 0s - loss: 0.0015 - acc: 0.9992 - val_loss: 14.6348 - val_acc: 0.0820
Epoch 99/10000
 - 0s - loss: 0.0011 - acc: 0.9992 - val_loss: 14.6348 - val_acc: 0.0820
Epoch 100/10000
 - 0s - loss: 0.0038 - acc: 0.9984 - val_loss: 14.6348 - val_acc: 0.0820
Epoch 101/10000
 - 0s - loss: 0.0268 - acc: 0.9913 - val_loss: 14.6327 - val_acc: 0.0820
Epoch 00101: early stopping
1582
9


C:\Users\tceve\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, input_dim=9, activation="relu", kernel_initializer="uniform")`


Train on 1265 samples, validate on 317 samples
Epoch 1/10000
 - 1s - loss: 0.4549 - acc: 0.7723 - val_loss: 2.3898 - val_acc: 0.2050
Epoch 2/10000
 - 0s - loss: 0.3512 - acc: 0.8427 - val_loss: 2.9461 - val_acc: 0.1640
Epoch 3/10000
 - 0s - loss: 0.3240 - acc: 0.8569 - val_loss: 2.8295 - val_acc: 0.1262
Epoch 4/10000
 - 0s - loss: 0.2894 - acc: 0.8688 - val_loss: 4.8912 - val_acc: 0.1451
Epoch 5/10000
 - 0s - loss: 0.2662 - acc: 0.8791 - val_loss: 6.7130 - val_acc: 0.0852
Epoch 6/10000
 - 0s - loss: 0.2698 - acc: 0.8791 - val_loss: 2.9655 - val_acc: 0.1735
Epoch 7/10000
 - 0s - loss: 0.2454 - acc: 0.8972 - val_loss: 5.6004 - val_acc: 0.1546
Epoch 8/10000
 - 0s - loss: 0.2394 - acc: 0.8957 - val_loss: 4.5496 - val_acc: 0.1388
Epoch 9/10000
 - 0s - loss: 0.2197 - acc: 0.9028 - val_loss: 5.4895 - val_acc: 0.1609
Epoch 10/10000
 - 0s - loss: 0.2124 - acc: 0.9107 - val_loss: 7.2415 - val_acc: 0.1356
Epoch 11/10000
 - 0s - loss: 0.2009 - acc: 0.9130 - val_loss: 7.6577 - val_acc: 0.1230
Epoch

Epoch 94/10000
 - 0s - loss: 0.0080 - acc: 0.9960 - val_loss: 12.4019 - val_acc: 0.1136
Epoch 95/10000
 - 0s - loss: 0.0065 - acc: 0.9945 - val_loss: 12.5277 - val_acc: 0.1136
Epoch 96/10000
 - 0s - loss: 0.0065 - acc: 0.9953 - val_loss: 12.6241 - val_acc: 0.1104
Epoch 97/10000
 - 0s - loss: 0.0059 - acc: 0.9968 - val_loss: 12.7166 - val_acc: 0.1073
Epoch 98/10000
 - 0s - loss: 0.0059 - acc: 0.9968 - val_loss: 12.5786 - val_acc: 0.1199
Epoch 99/10000
 - 0s - loss: 0.0057 - acc: 0.9960 - val_loss: 12.5850 - val_acc: 0.1167
Epoch 100/10000
 - 0s - loss: 0.0195 - acc: 0.9937 - val_loss: 12.3431 - val_acc: 0.1009
Epoch 101/10000
 - 0s - loss: 0.0088 - acc: 0.9968 - val_loss: 12.5329 - val_acc: 0.0946
Epoch 00101: early stopping
